In [1]:
from config import API_KEY, API_SECRET_KEY, BEARER_TOKEN, ACCESS_TOKEN, ACCESS_TOKEN_SECRET, EXCEPTIONS, SCREEN_NAME
import tweepy
import time

In [102]:
def connect():
    """Setting up the api connection"""
    auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)  # Read&Write Permissions
    return tweepy.API(auth)

In [103]:
def connection_verified(api):
    """Verifying twitter api connection"""
    print('Verifying connection..')
    try:
        api.verify_credentials()
        print('Verified')
        return True
    except:
        print('Not Verified...')
        return False

In [104]:
def get_followers_ids(handle, api):
    """Returns all the followers"""
    print('Getting followers\' ids')
    followers_iter = tweepy.Cursor(api.get_follower_ids, screen_name=handle).items()
    followers = set()
    while 1:
        try:
            user_id = next(followers_iter)
            followers.add(user_id)
            time.sleep(5)
        except tweepy.TweepyException as err:
            print(f'Exception occurred: {err}')
            time.sleep(60*15)
            continue
        except StopIteration:
            print('All the followers are extracted.')
            break
    return followers

In [105]:
def get_following(handle, api):
    print("Getting following's ids")
    following_iter = tweepy.Cursor(api.get_friends, screen_name=handle).items()
    while True:
        try:
            user = next(following_iter)
            yield user
        except tweepy.TweepyException as err:
            print(f'Exception occurred: {err}')
            time.sleep(60*15)
            continue
        except StopIteration:
            print('Iterator is exhausted...')
            break

In [106]:
def get_exceptions_ids(exceptions, api):
    """Returns set of followers who might not follow you back, but you still would like to follow them"""
    exceptions_ids = set()
    for handle in exceptions:
        user = api.get_user(screen_name=handle)
        exceptions_ids.add(user.id)
    return exceptions_ids

In [107]:
def our_friends(handle, exceptions, api):
    followers_ids = get_followers_ids(handle, api)
    exceptions_ids = get_exceptions_ids(exceptions, api)
    return followers_ids.union(exceptions_ids)

In [108]:
def is_loser(handle, friends, api):
    """Checks if retrieved user is in our friend list, if user is not it yields user's id and name."""
    losers = get_following(handle, api)
    try:
        loser = next(losers)
        if loser.id not in friends:
            print(f'Detected loser to unfollow {loser.name}')
            return loser
        else:
            return False
    except StopIteration:
        print('Looped through users.')
        return False

In [109]:
def unfollow_losers(handle, exceptions, api):
    """Unfollows users who do not follow you back except people who you want to keep following
    -handle: your twitter handle
    -exceptions: users who you want to keep following
    :return the dict of users who were unfollowed
    """
    friends = our_friends(handle, exceptions, api)
    unfollow_dict = {}
    while True:
        try:
            loser = is_loser(handle, friends, api)
            if not loser:
                time.sleep(10)
                continue
            else:
                loser.unfollow()
                unfollow_dict[loser.id] = loser.name
                print(f'{loser.name} is unfollowed.')
                time.sleep(10)
        except tweepy.TweepyException as err:
            print(f'Exception occurred: {err}')
        except StopIteration:
            print('Process finished.')
            print(f'Following list has been cleaned.')
        finally:
            print(f'{len(unfollow_dict)} has been unfollowed.')
            return unfollow_dict

In [110]:
def main():
    api = connect()
    if not connection_verified(api):
        return False
    unfollowed = unfollow_losers(SCREEN_NAME, EXCEPTIONS, api)
    return unfollowed

In [111]:
# main()